# Notebook 03: Entrenamiento y Optimización (OE3)

**Objetivo:** Entrenar un modelo ligero para clasificar estados afectivos.

## ⏱️ Estimación de Tiempos y Recursos

| Entorno | Hardware | Tiempo Estimado (MobileNetV3) | Nota |
| :--- | :--- | :--- | :--- |
| **Google Colab** | **GPU T4** | **30 min - 1.5 horas** | 🚀 **Recomendado** |
| **Local (PC)** | CPU | **12 - 24 horas** | ❌ Evitar |
| **Local (PC)** | GPU (GTX 1650+) | **1 - 3 horas** | ✅ Opción viable |


In [ ]:
!pip install tensorflow numpy matplotlib

In [ ]:
import os
import sys
import tensorflow as tf

# Configuración Híbrida (Drive vs Local)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    # AJUSTA ESTA RUTA A TU CARPETA EN DRIVE
    base_path = '/content/drive/MyDrive/Tesis_EdgeAI/Proy_Repo'
    if not os.path.exists(base_path):
         print(f"Ruta no encontrada: {base_path}. Verifica tu Drive.")
    else:
         os.chdir(base_path)
    print("Ejecutando en COLAB")
except ImportError:
    print("Ejecutando en LOCAL")

sys.path.append(os.path.abspath('.'))

In [ ]:
from src.models.architectures import build_mini_xception, build_mobilenet_v3_small
from src.models.train import train_model
from src.models.optimize import ModelOptimizer

In [ ]:
print("GPUs Disponibles:", tf.config.list_physical_devices('GPU'))

## 2. Selección de Estrategia de Entrenamiento
Tienes dos opciones. Ejecuta **SOLO UNA** de las siguientes celdas según tu elección.

### 🏆 Opción A: MobileNetV3 (Recomendado - Transfer Learning)
- **Ideal para:** Resultados rápidos, mayor precisión con pocos datos.
- **Tiempo:** ~30 mins en Colab.

In [ ]:
# OPCIÓN A: Entrenar MobileNetV3
history = train_model("data/raw/fer2013", model_type="mobilenet", epochs=20, batch_size=32)
model_to_save = "mobilenet"

### 🧪 Opción B: Mini-Xception (Entrenamiento desde Cero)
- **Ideal para:** Comparación académica, modelo ligero desde cero.
- **Tiempo:** ~1 hora en Colab (necesita más épocas).

In [ ]:
# OPCIÓN B: Entrenar Mini-Xception (Descomenta para usar)
# history = train_model("data/raw/fer2013", model_type="mini_xception", epochs=50, batch_size=32)
# model_to_save = "mini_xception"

## 3. Optimización para Edge (TFLite)
Convertimos el modelo seleccionado a formato ligero (.tflite) con cuantización INT8.

In [ ]:
# Asegurarse de que 'model_to_save' esté definido (por si ejecutaste la celda de arriba)
try:
    model_name = model_to_save
except NameError:
    model_name = "mobilenet" # Fallback por defecto

model_path = f"models/checkpoints/{model_name}_best.keras"

if os.path.exists(model_path):
    optimizer = ModelOptimizer(model_path)
    optimizer.convert_to_tflite(quantization='int8')
    print(f"Modelo optimizado {model_name} guardado en models/optimized/")
else:
    print("No se encontró el checkpoint para optimizar.")